# Building a Multi-Agent System

In this notebook, we showcase how the toolkit can be used to use a mixture of inbuilt tools and agents, as well as custom tools and workflows.

We created a simple [mixture-of-agents](./retail_sales_agent/) that serves as an assistant in retail sales. 
> **Note**: *This is just an example agent system that uses dummy data. The intention is to demonstrate some of the capabilities of this toolkit and how a new user can get familiar with it.* 

This agent system has:
1) A **supervisor** agent that routes incoming requests to the downstream agent expert
2) A **data insight** agent that is a tool-calling agent capable of answering questions about sales data
3) A **RAG agent** that is capable of answering questions about products using context from a product catalog
4) A **data visualization** agent that is capable of plotting graphs and trends

We demonstrate the following capabilities:
- RAG
- Multi-framework support
- Human-in-the-Loop
- Multi-agent support

For more capabilities, refer to the `examples` directory.

> **Note**: 
> All source code for this example can be found at [./retail_sales_agent](./retail_sales_agent/)

In [ ]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

if "TAVILY_API_KEY" not in os.environ:
    tavily_api_key = getpass.getpass("Enter your Tavily API key: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

if "OPENAI_API_KEY" not in os.environ:
    openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

### Creating a New Workflow for this Agent

To recap, to create a new workflow for this mixture of agents, we need to use the `nat workflow create` sub-command which creates the necessary directory structure. 

> **Note**: You can create this directory structure manually as well.

All new functions (tools and agents) that you want to be a part of this agent system can be created inside this directory for easier grouping of plugins. The only necessity for discovery by the toolkit is to import all new files/functions or simply define them in the `register.py` function.

The example referenced in this notebook has already been created in the [retail_sales_agent](./retail_sales_agent/) uisng the following command:
```bash
nat workflow create --workflow-dir . retail_sales_agent
```

### Adding Tools

To start off simple, let's create a single agent that serves as a helpful assistant that can answer questions about the retail sales CSV data. It will call tools to fetch daily sales of a product, calculate total sales per day and detect any outliers in sales.

**Function Creation**: All tools are created in [data_insight_tools.py](./retail_sales_agent/src/nat_retail_sales_agent/data_insight_tools.py). They each have a configuration object and the registered function.

**Import the registered function**: Make sure to import the registered function in [register.py](./retail_sales_agent/src/nat_retail_sales_agent/register.py)

**Create the YAML file**: For simplicity, we use the inbuilt react agent in the workflow and define the tools that should be made available to the agent. We also set the LLM to use. You can find the config file at [config.yml](./retail_sales_agent/configs/config.yml)

In [ ]:
!nat run --config_file retail_sales_agent/configs/config.yml --input "How do laptop sales compare to phone sales?"

Some other test queries that can be run are:
- "What were the laptop sales on Feb 16th 2024?"
- "What were the outliers in sales?"

### Adding a Retrieval Tool using Llamaindex

Next, let's add in a tool that is capable of performing retrieval of additional context to answer questions about products. It will use a vector store that stores details about products. We can create this agent using llama-index to demonstrate the framework-agnostic capability of the library. 

Refer to the code for the `product_catalog_rag` tool in [llama_index_rag_tool.py](./retail_sales_agent/src/nat_retail_sales_agent/llama_index_rag_tool.py). This can use a Milvus vector store for GPU-accelerated indexing. 

It requires the addition of an embedder section the [config_with_rag.yml](./retail_sales_agent/configs/config_with_rag.yml). This section follows a the same structure as the llms section and serves as a way to separate the embedding models from the LLM models. In our example, we are using the `nvidia/nv-embedqa-e5-v5` model.


You can test this workflow with the following command:

In [ ]:
!nat run --config_file retail_sales_agent/configs/config_with_rag.yml --input "What is the Ark S12 Ultra tablet and what are its specifications?"

### Adding Agents and a Supervisor Agent


Building on the previous workflow, we can create an example that shows how to build a `react_agent` serving as a master orchestrator that routes queries to specialized `tool_calling_agent` or `react_agent` experts based on query content and agent descriptions. Further, it will exemplify how complete agent workflows can be wrapped and used as tools by other agents, enabling complex multi-agent orchestration.

The full configuration file can be found at [config_multi_agent.yml](notebooks/retail_sales_agent/configs/config_multi_agent.yml)

```yaml
workflow:
  _type: react_agent
  tool_names: [data_analysis_agent, data_visualization_agent, rag_agent]
  llm_name: supervisor_llm
  verbose: true
  handle_parsing_errors: true
  max_retries: 2
  system_prompt: |
    Answer the following questions as best you can. You may communicate and collaborate with various experts to answer the questions:

    {tools}

    You may respond in one of two formats.
    Use the following format exactly to communicate with an expert:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action (if there is no required input, include "Action Input: None")
    Observation: wait for the expert to respond, do not assume the expert's response

    ... (this Thought/Action/Action Input/Observation can repeat N times.)
    Use the following format once you have the final answer:

    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
```

The above workflow sections shows how a supervisor agent can be defined that behaves as the orchestrator and routes to downstream experts based on their function descriptions. The experts in this example are the previously created `data_analysis_agent` and two new agents - `rag_agent` created to handle RAG using the retrieval tool and `data_visualization_agent` to create plots and visualizations of data as requested by the user.

In [ ]:
%%bash
nat run --config_file retail_sales_agent/configs/config_multi_agent.yml \
  --input "What is the Ark S12 Ultra tablet and what are its specifications?" \
  --input "How do laptop sales compare to phone sales?" \
  --input "Plot average daily revenue"

### Custom LangGraph Agent and Human-in-the-Loop

Besides using inbuilt agents in the workflows, we can also create custom agents using LangGraph or any other framework and bring them into a workflow. We demonstrate this by swapping out the `react_agent` used by the data visualization expert for a custom agent that has human-in-the-loop capability (utilizing a reusable plugin for HITL in the NeMo-Agent-Toolkit). The agent will ask the user whether they would like a summary of graph content.

The code can be found in [data_visualization_agent.py](examples/retail_sales_agent/src/nat_retail_sales_agent/data_visualization_agent.py)

This agent has an agent node, a tools node, a node to accept human input and a summarizer node.

Agent → generates tool calls → conditional_edge routes to tools

Tools → execute → edge routes back to data_visualization_agent

Agent → detects ToolMessage → creates summary AIMessage → conditional_edge routes to check_hitl_approval

HITL → approval → conditional_edge routes to summarize or end


#### Human-in-the-Loop Plugin

This is enabled by leveraging a reusable plugin developed in the [examples/HITL/por_to_jiratickets](../HITL/por_to_jiratickets/) example. We can view the implementation in the [nat_por_to_jiratickets.hitl_approval_tool.py](../HITL/por_to_jiratickets/src/nat_por_to_jiratickets/hitl_approval_tool.py) file. The implementation is shown below:

```python
@register_function(config_type=HITLApprovalFnConfig)
async def hitl_approval_function(config: HITLApprovalFnConfig, builder: Builder):

    import re

    prompt = f"{config.prompt} Please confirm if you would like to proceed. Respond with 'yes' or 'no'."

    async def _arun(unused: str = "") -> bool:

        nat_context = Context.get()
        user_input_manager = nat_context.user_interaction_manager

        human_prompt_text = HumanPromptText(text=prompt, required=True, placeholder="<your response here>")
        response: InteractionResponse = await user_input_manager.prompt_user_input(human_prompt_text)
        response_str = response.content.text.lower()  # type: ignore
        selected_option = re.search(r'\b(yes)\b', response_str)

        if selected_option:
            return True
        return False
        # Rest of the function
```

As we see above, requesting user input using NeMo Agent toolkit is straightforward. We can use the user_input_manager to prompt the user for input. The user's response is then processed to determine the next steps in the workflow. This can occur in any tool or function in the workflow, allowing for dynamic interaction with the user as needed.

Test the new workflow using the following command.

>**Note**: This command needs to be run in a terminal since it requires accepting human input. Please open a terminal and run this command.

```bash
nat run --config_file retail_sales_agent/configs/config_multi_agent_hitl.yml --input "Plot average daily revenue"
```

### Next Steps:

The above feature examples are not exhaustive. The NeMo-Agent-Toolkit supports a continuously expanding list of features like [long-term memory support](../frameworks/semantic_kernel_demo) through partner integrations, [Model Context Protocol compatibility](../MCP/simple_calculator_mcp), a [demo chat UI](examples/UI), [custom API routes](../front_ends/simple_calculator_custom_routes) and so on. Please refer to the [examples](../) directory for a full catalog of examples.

To package and distribute your agent, the process is straightforward and follows standard Python `pyproject.toml` packaging steps. Refer to [this documentation](https://docs.nvidia.com/nemo/agent-toolkit/latest/extend/sharing-components.html) for a more detailed guide.

Make sure to include all necessary NeMo Agent toolkit dependencies in the `pyproject.toml` as well as entrypoints.

You can use the `nat info components` to discover the dependencies that need to be included in the `pyproject.toml`.

Then you can either publish your package to a remote registry, build a wheel package for distribution, or share the source code.